In [62]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [63]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail_3'),
    spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'},
    default_reference='GRCh38'
) 

2023-07-28 13:47:16.915 Hail: WARN: Hail has already been initialized. If this call was intended to change configuration, close the session with hl.stop() first.


FatalError: IllegalArgumentException: requirement failed

Java stack trace:
java.lang.IllegalArgumentException: requirement failed
	at scala.Predef$.require(Predef.scala:268)
	at is.hail.backend.spark.SparkBackend$.apply(SparkBackend.scala:233)
	at is.hail.backend.spark.SparkBackend.apply(SparkBackend.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerCon

In [66]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
import numpy as np
hl.plot.output_notebook()
import bokeh
from scipy import stats

from bokeh.plotting import figure, output_notebook, show
output_notebook

Loading BokehJS ...

<function bokeh.io.output.output_notebook(resources=None, verbose=False, hide_banner=False, load_timeout=5000, notebook_type='jupyter')>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from oligogenic_model import (
    read_in_pheno_anno_all
)

In [ ]:
mt = read_in_pheno_anno_all(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt',
    'all')

2023-07-26 21:08:21.501 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-07-26 21:08:22.513 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)
2023-07-26 21:08:23.807 Hail: INFO: Reading table to impute column types
2023-07-26 21:08:24.820 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading

In [11]:
mt = hl.read_matrix_table('/localfs/4130551/all.mt')

In [12]:
samples_to_exclude = ['S_7288', 'S_7289', 'S_7290', 'WGS_6827', 'WGS_6835', 'B502', 'B506']

In [13]:
mt = mt.filter_cols(
    hl.literal(samples_to_exclude).contains(mt.s),
    keep = False
)

In [14]:
mt.count()

(6538550, 280)

In [15]:
mt.aggregate_cols(hl.agg.counter(mt.group))

{'GTS': 180, 'local_controls': 100}

In [16]:
gts = mt.filter_cols(mt.group == 'GTS')
sport = mt.filter_cols(mt.group == 'local_controls')

In [17]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [18]:
mt = mt.naive_coalesce(500)

In [19]:
gts = gts.filter_rows(hl.agg.any(gts.GT.is_non_ref()))

In [20]:
gts = gts.naive_coalesce(500)

In [21]:
sport = sport.filter_rows(hl.agg.any(sport.GT.is_non_ref()))

In [22]:
sport = sport.naive_coalesce(500)

In [23]:
mt.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt',
    overwrite = True
)

2023-07-27 08:21:20.712 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt


In [7]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt')

In [54]:
mt.aggregate_rows(
    hl.agg.counter(
        mt.vep.vep.most_severe_consequence
    )
)

{'3_prime_UTR_variant': 332008,
 '5_prime_UTR_variant': 95013,
 'TF_binding_site_variant': 7945,
 'coding_sequence_variant': 17,
 'downstream_gene_variant': 224909,
 'incomplete_terminal_codon_variant': 27,
 'intergenic_variant': 273782,
 'intron_variant': 4355912,
 'mature_miRNA_variant': 363,
 'missense_variant': 236333,
 'non_coding_transcript_exon_variant': 269293,
 'regulatory_region_variant': 104832,
 'splice_acceptor_variant': 1975,
 'splice_donor_variant': 3094,
 'splice_region_variant': 30457,
 'start_lost': 587,
 'stop_gained': 6538,
 'stop_lost': 903,
 'stop_retained_variant': 261,
 'synonymous_variant': 251561,
 'upstream_gene_variant': 318946}

In [10]:
mt.count()

(6514756, 280)

In [13]:
mt = hl.variant_qc(mt)
mt = hl.sample_qc(mt)
mt = mt.checkpoint(localfs_path+'mt-qced.mt')

2023-07-28 10:11:04.488 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /localfs/4138335/mt-qced.mt


In [65]:
mt = hl.read_matrix_table('/localfs/4138335/mt-qced.mt')

In [67]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.98,1), legend='Call Rate')
show(p)

In [69]:
p2 = hl.plot.histogram(mt.variant_qc.dp_stats.mean, range=(0,100), legend='per variant coverage')
show(p2)

In [24]:
mt.aggregate_cols(hl.agg.stats(mt.sample_qc.gq_stats.mean))

Struct(mean=75.93978638203325, stdev=13.583481489040917, min=13.385729516374342, max=97.20849875610891, n=280, sum=21263.14018696931)

In [55]:
mt.aggregate_cols(hl.agg.stats(mt.sample_qc.dp_stats.mean))

Struct(mean=29.730013283546572, stdev=7.360488628074394, min=6.627429804620724, max=57.15210555826873, n=280, sum=8324.40371939304)

In [59]:
mt.aggregate_rows(hl.agg.stats(mt.variant_qc.gq_stats.mean))

Struct(mean=75.93623586649194, stdev=6.966540687133007, min=40.36071428571429, max=99.0, n=6514756, sum=494706048.2286436)

In [61]:
mt.aggregate_rows(hl.agg.stats(mt.variant_qc.dp_stats.mean))

Struct(mean=29.728882857576213, stdev=20.614663891685222, min=11.767857142857142, max=3693.1035714285713, n=6514756, sum=193676417.96969178)

In [ ]:
gts #check the duplicated sample!!!

In [71]:
gts.count()

(5209343, 180)

In [74]:
gts.cols().show(180)

+-------------+-------------------------+--------------------------+
| s           | sample_qc.dp_stats.mean | sample_qc.dp_stats.stdev |
+-------------+-------------------------+--------------------------+
| str         |                 float64 |                  float64 |
+-------------+-------------------------+--------------------------+
| "460"       |                2.46e+01 |                 1.12e+01 |
| "461"       |                2.51e+01 |                 6.88e+00 |
| "462"       |                2.49e+01 |                 6.66e+00 |
| "463"       |                2.18e+01 |                 6.31e+00 |
| "465"       |                2.81e+01 |                 7.88e+00 |
| "466"       |                2.66e+01 |                 7.41e+00 |
| "467"       |                2.49e+01 |                 6.77e+00 |
| "468"       |                2.91e+01 |                 6.85e+00 |
| "469"       |                2.98e+01 |                 7.00e+00 |
| "470"       |                2.20e+01 |                 6.17e+00 |
| "470"       |                2.20e+01 |                 6.17e+00 |
| "471"       |                2.44e+01 |                 6.85e+00 |
| "472"       |                2.48e+01 |                 7.20e+00 |
| "473"       |                2.80e+01 |                 6.70e+00 |
| "474"       |                2.47e+01 |                 6.75e+00 |
| "475"       |                2.46e+01 |                 6.50e+00 |
| "476"       |                2.36e+01 |                 6.70e+00 |
| "477"       |                1.93e+01 |                 6.21e+00 |
| "478"       |                2.40e+01 |                 6.41e+00 |
| "479"       |                2.90e+01 |                 8.11e+00 |
| "480"       |                2.37e+01 |                 6.59e+00 |
| "481"       |                2.92e+01 |                 7.97e+00 |
| "482"       |                2.74e+01 |                 7.15e+00 |
| "483"       |                2.44e+01 |                 6.87e+00 |
| "484"       |                2.49e+01 |                 6.41e+00 |
| "485"       |                2.74e+01 |                 7.38e+00 |
| "486"       |                2.66e+01 |                 7.57e+00 |
| "487"       |                3.08e+01 |                 7.23e+00 |
| "488"       |                2.78e+01 |                 7.47e+00 |
| "489"       |                3.07e+01 |                 8.18e+00 |
| "490"       |                2.50e+01 |                 6.45e+00 |
| "491"       |                2.45e+01 |                 6.64e+00 |
| "492"       |                2.72e+01 |                 7.57e+00 |
| "493"       |                2.46e+01 |                 6.93e+00 |
| "S_136"     |                2.79e+01 |                 5.19e+01 |
| "S_170c"    |                2.86e+01 |                 4.30e+01 |
| "S_170d"    |                3.85e+01 |                 6.33e+01 |
| "S_6981"    |                2.55e+01 |                 3.99e+01 |
| "S_6982"    |                2.67e+01 |                 4.52e+01 |
| "S_7146"    |                3.93e+01 |                 5.54e+01 |
| "S_7156"    |                3.11e+01 |                 4.84e+01 |
| "S_7212"    |                3.39e+01 |                 5.59e+01 |
| "S_7213"    |                3.32e+01 |                 5.67e+01 |
| "S_7214"    |                3.28e+01 |                 5.14e+01 |
| "S_7227"    |                3.50e+01 |                 5.29e+01 |
| "S_7229"    |                3.13e+01 |                 3.92e+01 |
| "S_7231"    |                2.63e+01 |                 4.44e+01 |
| "S_7234"    |                2.71e+01 |                 4.37e+01 |
| "S_7236"    |                2.56e+01 |                 4.34e+01 |
| "S_7237"    |                3.48e+01 |                 5.86e+01 |
| "S_7238"    |                2.79e+01 |                 4.90e+01 |
| "S_7239"    |                2.81e+01 |                 4.24e+01 |
| "S_7240"    |                2.76e+01 |                 4.75e+01 